In [44]:
from Features.birds import *
from Features.sweden_map import *
from Features.weather import *
import pandas as pd
from datetime import datetime, timedelta


In [8]:
golden = API_bird_data(bird_type="goleag")
#white = API_bird_data(bird_type="whteag")



Bird data API response code: 200


In [9]:
print(golden)

[{'speciesCode': 'goleag', 'comName': 'Golden Eagle', 'sciName': 'Aquila chrysaetos', 'locId': 'L53102821', 'locName': 'Myrse yard', 'obsDt': '2025-11-29 11:50', 'howMany': 1, 'lat': 57.5359277, 'lng': 18.2807413, 'obsValid': True, 'obsReviewed': False, 'locationPrivate': True, 'subId': 'S286591513'}]


In [2]:
#print(white[0])
lat = 57.7358823
lng = 13.4065944

In [3]:
map=SwedenMap()

print(map.point_to_region(lat, lng))

Gotland


In [4]:
map.middle_point_of_region("Gotland")

(57.51723947372926, 18.538432396771)

In [5]:
API_tomorrow_weather(lon=lng, lat=lat, days=7)

{'latitude': 57.732597,
 'longitude': 13.398376,
 'generationtime_ms': 0.20515918731689453,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/Berlin',
 'timezone_abbreviation': 'GMT+1',
 'elevation': 165.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_mean': '°C',
  'precipitation_sum': 'mm',
  'weather_code': 'wmo code',
  'wind_speed_10m_mean': 'km/h'},
 'daily': {'time': ['2025-12-11',
   '2025-12-12',
   '2025-12-13',
   '2025-12-14',
   '2025-12-15',
   '2025-12-16',
   '2025-12-17'],
  'temperature_2m_mean': [7.8, 6.8, 5.0, 6.3, 6.1, 4.9, 3.3],
  'precipitation_sum': [13.1, 0.0, 2.58, 4.5, 3.4, 0.0, 0.0],
  'weather_code': [63, 3, 51, 51, 51, 3, 3],
  'wind_speed_10m_mean': [18.0, 8.0, 12.8, 19.9, 17.8, 13.5, 13.0]}}

In [6]:
historical_weather_download(start_date="2025-12-01", lon=lng, lat=lat)

{'latitude': 57.75044,
 'longitude': 13.218143,
 'generationtime_ms': 8.298039436340332,
 'utc_offset_seconds': 3600,
 'timezone': 'Europe/Berlin',
 'timezone_abbreviation': 'GMT+1',
 'elevation': 165.0,
 'daily_units': {'time': 'iso8601',
  'temperature_2m_mean': '°C',
  'precipitation_sum': 'mm',
  'wind_speed_10m_mean': 'km/h',
  'weather_code': 'wmo code'},
 'daily': {'time': ['2025-12-01',
   '2025-12-02',
   '2025-12-03',
   '2025-12-04',
   '2025-12-05',
   '2025-12-06',
   '2025-12-07',
   '2025-12-08',
   '2025-12-09'],
  'temperature_2m_mean': [4.5, 3.9, 3.0, 3.4, 3.1, 3.6, 4.1, 6.4, 6.6],
  'precipitation_sum': [1.0, 1.6, 1.1, 2.0, 1.6, 2.3, 1.9, 18.4, 3.7],
  'wind_speed_10m_mean': [20.7, 20.1, 10.6, 10.8, 10.4, 9.7, 12.8, 22.5, 15.9],
  'weather_code': [51, 51, 51, 51, 53, 53, 71, 63, 53]}}

## Bird data

What we want: 
lat, lon, num obs, time (maybe), obsReviewed

from api:


In [58]:
df1 = pd.read_csv(
    'Data/ebd_SE_goleag_smp_relOct-2025_extracted/ebd_SE_goleag_smp_relOct-2025_sampling.txt', 
    sep='\t', 
    low_memory=False, # Prevents DtypeWarnings on mixed-type columns in large files
    parse_dates=['OBSERVATION DATE', 'LAST EDITED DATE'] # Optional: converts dates immediately
)

df2 = pd.read_csv(
    "Data/ebd_SE_goleag_smp_relOct-2025_extracted/ebd_SE_goleag_smp_relOct-2025.txt",
    sep='\t', 
    low_memory=False, # Prevents DtypeWarnings on mixed-type columns in large files
    parse_dates=['OBSERVATION DATE', 'LAST EDITED DATE'] # Optional: converts dates immediately
)

print(df1.shape)
print(df2.shape)

(418599, 34)
(2237, 53)


In [ ]:
df1

In [ ]:
df2

In [ ]:
#1
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols_to_drop = ['LAST EDITED DATE', 'COUNTRY', 'COUNTRY CODE', 'STATE', 'STATE CODE',
       'COUNTY', 'COUNTY CODE', 'IBA CODE', 'BCR CODE', 'USFWS CODE',
       'ATLAS BLOCK', 'LOCALITY', 'LOCALITY ID', 'LOCALITY TYPE', 'OBSERVER ID', 'OBSERVER ORCID ID', 'OBSERVATION TYPE', 'PROTOCOL NAME', 'PROTOCOL CODE', 'PROJECT NAMES',
       'PROJECT IDENTIFIERS', 'DURATION MINUTES', 'EFFORT DISTANCE KM','EFFORT AREA HA', 'GROUP IDENTIFIER', 'Unnamed: 33', "NUMBER OBSERVERS", "SAMPLING EVENT IDENTIFIER"]

    return df.drop(columns=cols_to_drop)

# TODO: merge the bird dfs and add bird species column

#2 merge dataframe with daily observation

def merge_bird_with_daily(bird_df: pd.DataFrame, today:str, b_type:str) -> pd.DataFrame:
   daily_dict=API_bird_data(bird_type=b_type)
   daily_df = pd.DataFrame(daily_dict)
   
   daily_df.rename(columns={"obsDt":"OBSERVATION DATE", "lat": "LATITUDE", "lng": "LONGITUDE"}, inplace=True)
   daily_df.drop(columns=["speciesCode","comName", "sciName", "locId", "locName", "obsValid", "obsReviewed", "locationPrivate", "subId"], inplace=True) # what is the howMany column?

   
   # todo split time and date
   

   return daily_df


In [54]:
TODAY = datetime.now().strftime("%Y-%m-%d")
print("Today's date:", TODAY)
df_dropped = drop_unused_columns(df1)
#daily_df = merge_bird_with_daily(df_dropped, today=TODAY, b_type="whteag")
#daily_df=merge_bird_with_daily(df_dropped, today=TODAY, b_type="goleag")



Today's date: 2025-12-11


In [55]:
df_dropped.head()

,LATITUDE,LONGITUDE,OBSERVATION DATE,TIME OBSERVATIONS STARTED,ALL SPECIES REPORTED,CHECKLIST COMMENTS
0,55.723208,13.222415,2023-11-28,10:22:00,1,"I tried to see the nutcracker, but it didn't s..."
1,58.556193,13.866578,2023-11-28,12:35:00,0,NaN
2,55.447491,12.948929,2023-11-26,10:25:00,1,NaN
3,55.455841,12.976714,2023-11-26,10:57:00,1,NaN
4,58.558809,13.892296,2023-11-30,15:51:00,1,NaN
